In [1]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("RetailSalesAnalysis").master('local').getOrCreate()

df_shopping_data = spark.read.csv("customer_shopping_data.csv", header=True, inferSchema=True)
df_shopping_data.show()

+----------+-----------+------+---+---------------+--------+-------+--------------+------------+-----------------+
|invoice_no|customer_id|gender|age|       category|quantity|  price|payment_method|invoice_date|    shopping_mall|
+----------+-----------+------+---+---------------+--------+-------+--------------+------------+-----------------+
|   I138884|    C241288|Female| 28|       Clothing|       5| 1500.4|   Credit Card|    5/8/2022|           Kanyon|
|   I317333|    C111565|  Male| 21|          Shoes|       3|1800.51|    Debit Card|  12/12/2021|   Forum Istanbul|
|   I127801|    C266599|  Male| 20|       Clothing|       1| 300.08|          Cash|   9/11/2021|        Metrocity|
|   I173702|    C988172|Female| 66|          Shoes|       5|3000.85|   Credit Card|  16/05/2021|     Metropol AVM|
|   I337046|    C189076|Female| 53|          Books|       4|   60.6|          Cash|  24/10/2021|           Kanyon|
|   I227836|    C657758|Female| 28|       Clothing|       5| 1500.4|   Credit Ca

In [8]:
df_shopping_data.createOrReplaceTempView('shopping_data')
df_shopping_data.printSchema()

query = """
SELECT category, COUNT(*) AS total_sales
FROM shopping_data
GROUP BY category
"""

result_df = spark.sql(query)
result_df.show()


root
 |-- invoice_no: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- invoice_date: string (nullable = true)
 |-- shopping_mall: string (nullable = true)



+---------------+-----------+
|       category|total_sales|
+---------------+-----------+
|Food & Beverage|      14776|
|       Clothing|      34487|
|       Souvenir|       4999|
|          Books|       4981|
|     Technology|       4996|
|      Cosmetics|      15097|
|          Shoes|      10034|
|           Toys|      10087|
+---------------+-----------+



In [ ]:
df_shopping_data.select("invoice_date").show(5)

# Convert string date to proper date format
from pyspark.sql.functions import to_date, month, quarter, year, when, coalesce

# Handle mixed date formats
df_with_dates = df_shopping_data.withColumn("date", 
    coalesce(
        to_date(df_shopping_data.invoice_date, "d/M/yyyy"),
        to_date(df_shopping_data.invoice_date, "M/d/yyyy")
    )
)

# Show the converted dates
df_with_dates.select("invoice_date", "date").show(5)

+------------+
|invoice_date|
+------------+
|    5/8/2022|
|  12/12/2021|
|   9/11/2021|
|  16/05/2021|
|  24/10/2021|
+------------+
only showing top 5 rows
+------------+----------+
|invoice_date|      date|
+------------+----------+
|    5/8/2022|2022-08-05|
|  12/12/2021|2021-12-12|
|   9/11/2021|2021-11-09|
|  16/05/2021|2021-05-16|
|  24/10/2021|2021-10-24|
+------------+----------+
only showing top 5 rows
+------------+----------+
|invoice_date|      date|
+------------+----------+
|    5/8/2022|2022-08-05|
|  12/12/2021|2021-12-12|
|   9/11/2021|2021-11-09|
|  16/05/2021|2021-05-16|
|  24/10/2021|2021-10-24|
+------------+----------+
only showing top 5 rows


In [ ]:
# Add seasonal columns
df_seasonal = (
    df_with_dates
    .withColumn("year", year("date"))                      
    .withColumn("month", month("date"))                    
    .withColumn("quarter", quarter("date"))                
    .withColumn(
        "season",                                          
        when(month("date").isin([12, 1, 2]), "Winter")     
        .when(month("date").isin([3, 4, 5]), "Spring")     
        .when(month("date").isin([6, 7, 8]), "Summer")     
        .otherwise("Fall")                                 
    )
)

df_seasonal.createOrReplaceTempView('seasonal_data')



In [13]:
# Monthly Sales Trends
monthly_query = """
SELECT 
    year,                                      
    month,                                     
    COUNT(*) as transaction_count,             
    SUM(quantity) as total_quantity,           
    SUM(price * quantity) as total_revenue,    
    AVG(price * quantity) as avg_transaction_value 
FROM seasonal_data 
WHERE date IS NOT NULL                         
GROUP BY year, month                           
ORDER BY year, month                           
"""

monthly_trends = spark.sql(monthly_query)
print("Monthly Trends:")
monthly_trends.show(24)  

Monthly Trends:
+----+-----+-----------------+--------------+--------------------+---------------------+
|year|month|transaction_count|total_quantity|       total_revenue|avg_transaction_value|
+----+-----+-----------------+--------------+--------------------+---------------------+
|2021|    1|             3835|         11483|   9641614.620000066|   2514.1107222946716|
|2021|    2|             3407|         10179|   8772315.220000051|    2574.791670091004|
|2021|    3|             3813|         11383|   9455359.380000075|   2479.7690479937255|
|2021|    4|             3724|         11270|    9389541.54000008|   2521.3591675617827|
|2021|    5|             3848|         11620|    9771756.97000006|   2539.4378820166476|
|2021|    6|             3783|         11352|   9286271.350000052|   2454.7373380914755|
|2021|    7|             3984|         11986|  1.03111196800001E7|    2588.132449799222|
|2021|    8|             3723|         11205|   9630655.700000076|   2586.7998119796066|
|2021

In [14]:
# Seasonal Analysis
seasonal_query = """
SELECT 
    season,
    COUNT(*) as transaction_count,
    SUM(quantity) as total_quantity,
    SUM(price * quantity) as total_revenue,
    AVG(price * quantity) as avg_transaction_value
FROM seasonal_data 
WHERE date IS NOT NULL
GROUP BY season
ORDER BY 
    CASE season 
        WHEN 'Spring' THEN 1
        WHEN 'Summer' THEN 2
        WHEN 'Fall' THEN 3
        WHEN 'Winter' THEN 4
    END
"""

seasonal_trends = spark.sql(seasonal_query)
print("Seasonal Trends:")
seasonal_trends.show()

Seasonal Trends:
+------+-----------------+--------------+-------------------+---------------------+
|season|transaction_count|total_quantity|      total_revenue|avg_transaction_value|
+------+-----------------+--------------+-------------------+---------------------+
|Spring|            23914|         71999| 6.03912084099987E7|   2525.3495195282553|
|Summer|            23093|         69294|5.859485921999889E7|   2537.3428839907715|
|  Fall|            22680|         68161|5.754802528999865E7|   2537.3908858024097|
|Winter|            29770|         89258|7.497170132999755E7|   2518.3641696337772|
+------+-----------------+--------------+-------------------+---------------------+

+------+-----------------+--------------+-------------------+---------------------+
|season|transaction_count|total_quantity|      total_revenue|avg_transaction_value|
+------+-----------------+--------------+-------------------+---------------------+
|Spring|            23914|         71999| 6.03912084099987

In [15]:
# Category-Specific Seasonal Patterns
category_seasonal_query = """
SELECT 
    category,
    season,
    COUNT(*) as transaction_count,
    SUM(price * quantity) as total_revenue,
    AVG(price * quantity) as avg_transaction_value
FROM seasonal_data 
WHERE date IS NOT NULL
GROUP BY category, season
ORDER BY category, 
    CASE season 
        WHEN 'Spring' THEN 1
        WHEN 'Summer' THEN 2
        WHEN 'Fall' THEN 3
        WHEN 'Winter' THEN 4
    END
"""

category_seasonal = spark.sql(category_seasonal_query)
print("Category-Specific Seasonal Patterns:")
category_seasonal.show(50)

Category-Specific Seasonal Patterns:
+---------------+------+-----------------+--------------------+---------------------+
|       category|season|transaction_count|       total_revenue|avg_transaction_value|
+---------------+------+-----------------+--------------------+---------------------+
|          Books|Spring|             1218|  198980.10000000027|    163.3662561576357|
|          Books|Summer|             1144|  196647.00000000015|    171.8942307692309|
|          Books|  Fall|             1107|  182966.54999999996|    165.2814363143631|
|          Books|Winter|             1512|  255959.24999999988|   169.28521825396817|
|       Clothing|Spring|             8339| 2.786722927999941E7|    3341.795092936732|
|       Clothing|Summer|             7979|2.6282506799999498E7|   3293.9599949867775|
|       Clothing|  Fall|             7880|2.6066449199999392E7|   3307.9250253806335|
|       Clothing|Winter|            10289|3.3780605759998865E7|    3283.176767421408|
|      Cosmetics|

In [16]:
# Year-over-Year Seasonal Comparison
yoy_seasonal_query = """
SELECT 
    year,
    season,
    COUNT(*) as transaction_count,
    SUM(price * quantity) as total_revenue,
    LAG(SUM(price * quantity)) OVER (PARTITION BY season ORDER BY year) as prev_year_revenue,
    ROUND(
        ((SUM(price * quantity) - LAG(SUM(price * quantity)) OVER (PARTITION BY season ORDER BY year)) 
         / LAG(SUM(price * quantity)) OVER (PARTITION BY season ORDER BY year)) * 100, 2
    ) as yoy_growth_percent
FROM seasonal_data 
WHERE date IS NOT NULL
GROUP BY year, season
ORDER BY season, year
"""

yoy_comparison = spark.sql(yoy_seasonal_query)
print("Year-over-Year Seasonal Comparison:")
yoy_comparison.show()

Year-over-Year Seasonal Comparison:
+----+------+-----------------+--------------------+--------------------+------------------+
|year|season|transaction_count|       total_revenue|   prev_year_revenue|yoy_growth_percent|
+----+------+-----------------+--------------------+--------------------+------------------+
|2021|  Fall|            11384|2.8716735970000252E7|                NULL|              NULL|
|2022|  Fall|            11296|2.8831289320000228E7|2.8716735970000252E7|               0.4|
|2021|Spring|            11385| 2.861665789000035E7|                NULL|              NULL|
|2022|Spring|            11559|2.9260403730000317E7| 2.861665789000035E7|              2.25|
|2023|Spring|              970|          2514146.79|2.9260403730000317E7|            -91.41|
|2021|Summer|            11490| 2.922804673000024E7|                NULL|              NULL|
|2022|Summer|            11603|2.9366812490000248E7| 2.922804673000024E7|              0.47|
|2021|Winter|            11123|2.7